In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.Descriptors import CalcMolDescriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator as MDC

from sklearn.feature_selection import VarianceThreshold

# Download data

In [ ]:
target = 'aa2ar'
url = f'https://dude.docking.org//targets/{target}'
data_path = './data'
path_model = './models'
pics = './images'

In [ ]:
def dir_builder():
    if not os.path.exists(path_model):
        os.mkdir(path_model)
    if not os.path.exists(pics):
        os.mkdir(pics)
    if not os.path.exists(data):
        os.mkdir(data)

dir_builder()

In [ ]:
active = pd.read_csv(f'{url}/actives_combined.ism', header=None, sep=' ', on_bad_lines='skip')
inactive = pd.read_csv(f'{url}/inactives_combined.ism', header=None, sep=' ', on_bad_lines='skip')
decoys = pd.read_csv(f'{url}/decoys_final.ism', header=None, sep=' ', on_bad_lines='skip')

In [ ]:
active = pd.DataFrame({'smiles':active[0], 'label': [1]*len(active[0])})
active_size = active.shape[0]

inactive = pd.DataFrame({'smiles': inactive[0], 'label': [0]*len(inactive[0])})
inactive_size = inactive.shape[0]

decoys = pd.DataFrame({'smiles':decoys[0], 'label': [0]*len(decoys[0])})
decoys_size = decoys.shape[0]

# Feature generation

In [ ]:
def RDkit_descriptors(data):
    mols = [Chem.MolFromSmiles(i) for i in data.smiles]
    desc_names = list(CalcMolDescriptors(mols[0]).keys())
    desc_names.append("label")
    mol_descs = []
    for i in tqdm(range(len(data))):
        descriptors = CalcMolDescriptors(mols[i])
        descriptors['label'] = data.label[i]
        mol_descs.append(descriptors)
    return mol_descs, desc_names

In [ ]:
mol_classes = [active, inactive, decoys]
classes_names = ['active', 'inactive', 'decoys']

In [ ]:
for mol_class, class_name in zip(mol_classes, classes_names):
    print("Uploading", class_name, "dataset")
    mol_descs, desc_names = RDkit_descriptors(mol_class)
    descriptors_df = pd.DataFrame(mol_descs, columns=desc_names)
    descriptors_df.to_csv(f'{data_path}/{class_name}_descs_raw.csv', index=False)

# Data preparation

In [ ]:
active = pd.read_csv(f'{data_path}/active_descs_raw.csv', index_col=None)
active_size = active.shape[0]

inactive = pd.read_csv(f'{data_path}/inactive_descs_raw.csv', index_col=None)
inactive_size = inactive.shape[0]

decoys = pd.read_csv(f'{data_path}/decoys_descs_raw.csv', index_col=None)
decoys_size = decoys.shape[0]

In [ ]:
colors = sns.color_palette('pastel')[0:5]
data = [active_size, inactive_size, decoys_size]
plt.pie(data, labels=classes_names, colors = colors, autopct='%.0f%%', textprops={"fontsize":12})
plt.title('Raw data')
# plt.show()
plt.savefig(f'{pics}/raw_data.png')

# Data resampling

In [ ]:
decoys_new_size = active_size - inactive_size
split_decoys = decoys.sample(n=decoys_new_size)

balanced_data = pd.concat([active, inactive, split_decoys], ignore_index=True)

In [ ]:
balanced_data = pd.read_csv(f'{data_path}/balanced_dataframe.csv')

### Filtering - deleting invariant data and highly correlated data

In [ ]:
def filter_data(X,correlation_threshold = 0.9, variance_threshold = 0.9):
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(~np.tril(np.ones(corr_matrix.shape)).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > correlation_threshold)]

    column_names = X.columns
    X = X.drop(to_drop, axis=1)

    sel = VarianceThreshold(threshold=(variance_threshold * (1 - variance_threshold)))
    X = sel.fit_transform(X)
    kept_features = sel.get_support(indices=True)
    X = pd.DataFrame(X, columns=column_names[kept_features])

    return X

In [ ]:
X = balanced_data.iloc[:,:-1]
y = balanced_data.label
X = filter_data(X)
balanced_data = pd.concat([X, y], axis=1)

In [ ]:
balanced_data.to_csv(f'{data_path}/balanced_dataframe.csv',index=False)

In [ ]:
colors = sns.color_palette('pastel')[0:5]
data = [active_size, inactive_size, decoys_new_size]
plt.pie(data, labels=classes_names, colors = colors, autopct='%.0f%%', textprops={"fontsize":12})
plt.title('Balanced data')
# plt.show()
plt.savefig(f'{pics}/balanced_data.png')